# Battle City RL - DreamerV3 Training

This notebook sets up the environment to train a **DreamerV3** agent on **Battle City (NES)**.

## Instructions
1. **Upload Files**: Drag and drop the following files from your local project into the Colab file explorer (sidebar on the left):
    - `BattleCity.nes` (The ROM file)
    - `battle_city_env.py`
    - `config.py`
    - `dreamer_wrapper.py`
    - `train_dreamer.py`
2. **Runtime**: Ensure you are using a GPU Runtime (`Runtime` > `Change runtime type` > `T4 GPU` or better).
3. **Run All**: Run the cells below sequentially.

### 1. Install Dependencies
We need `nes-py` (and its build deps), `gymnasium`, `jax`, and `dreamerv3`.

In [ ]:
# 1. System Dependencies for NES emulator
!apt-get update > /dev/null
!apt-get install -y freeglut3-dev > /dev/null

# 2. Python Packages
# DreamerV3 and Embodied from GitHub (Official)
!pip install git+https://github.com/danijar/embodied.git
!pip install git+https://github.com/danijar/dreamerv3.git

# Environment & Utils
!pip install gym gymnasium nes-py opencv-python matplotlib

# 3. JAX (GPU)
# Colab usually has JAX pre-installed, but updating is good practice
!pip install -U "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

### 2. Verify Files
Ensure all necessary files are present.

In [ ]:
import os
required_files = ['BattleCity.nes', 'battle_city_env.py', 'config.py', 'dreamer_wrapper.py', 'train_dreamer.py']
missing = [f for f in required_files if not os.path.exists(f)]

if missing:
    print(f"❌ MISSING FILES: {missing}")
    print("Please upload them to the root directory (/content/).")
else:
    print("✅ All files found!")

### 3. Start Training
This launches the training script. 
- Check `logs/dreamer` for TensorBoard logs.
- Training will take hours to show significant progress, but Dreamer learns fast.
- Use `TensorBoard` extension to view graphs.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/dreamer

In [ ]:
!python train_dreamer.py